In [1]:
from pathlib import Path

from ray import tune
from ray.air import Checkpoint, session

In [2]:
ray_results = Path("~/ray_results").expanduser()
assert ray_results.is_dir()
last_experiment = max(ray_results.glob('_aux_train_daostack_*'), key=lambda x: x.stat().st_ctime)
print(f"Loading results from {last_experiment}...")
assert last_experiment.is_dir()

def _aux_train_daostack(*args, **kwargs):
    raise NotImplementedError

# TODO: Create an experiment grid or whatever is called
tuner = tune.Tuner.restore(str(last_experiment), _aux_train_daostack)
rg = tuner.get_results()
rg.get_best_result()

Loading results from /home/davo/ray_results/_aux_train_daostack_2023-08-17_16-06-29...


2023-08-17 16:18:21,348	INFO experiment_analysis.py:972 -- No trial data passed in during `ExperimentAnalysis` initialization -- you are most likely loading the experiment after it has completed.
Loading trial data from the experiment checkpoint file. This may result in loading some stale information, since checkpointing is periodic.


Result(
  metrics={'loss': 0.3653228515531959, 'val_loss': 0.5160669684410095, 'rprec train': 0.3332699239253998, 'rprec test': 0.1533869057893753, 'p@5 train': 0.4307692348957062, 'p@5 test': 0.20192307233810425, 'r@5 train': 0.0751161053776741, 'r@5 test': 0.1165977194905281, 'should_checkpoint': True, 'done': True, 'trial_id': '8c470f7f', 'experiment_tag': '1_trial_index=0,batch_size=6,conv_layers=5,embedding_dim=90,l2=0.0000,learning_rate=0.0001,max_epochs=50'},
  path='/home/davo/ray_results/_aux_train_daostack_2023-08-17_16-06-29/_aux_train_daostack_8c470f7f_1_trial_index=0,batch_size=6,conv_layers=5,embedding_dim=90,l2=0.0000,learning_rate=0.0001,max_epochs_2023-08-17_16-06-32',
  checkpoint=Checkpoint(local_path=/home/davo/ray_results/_aux_train_daostack_2023-08-17_16-06-29/_aux_train_daostack_8c470f7f_1_trial_index=0,batch_size=6,conv_layers=5,embedding_dim=90,l2=0.0000,learning_rate=0.0001,max_epochs_2023-08-17_16-06-32/checkpoint_000049)
)

In [3]:
# Read dataframe and drop some unneeded columns
rdf = rg.get_dataframe('p@5 test', 'max').drop(columns=['timestamp', 'node_ip', 'pid', 'hostname', 'time_since_restore', 'iterations_since_restore', 'logdir'])
rdf = rdf.drop(columns=['date'])
# rdf = rdf[rdf['done']]
print(rdf.columns)

for c in ['config/batch_size']: #, 'config/embedding_dim']:
    rdf[c] = 2**rdf[c]

rdf.sort_values('loss').head(5)

Index(['loss', 'val_loss', 'rprec train', 'rprec test', 'p@5 train',
       'p@5 test', 'r@5 train', 'r@5 test', 'time_this_iter_s',
       'should_checkpoint', 'done', 'training_iteration', 'trial_id',
       'time_total_s', 'config/__trial_index__', 'config/batch_size',
       'config/conv_layers', 'config/embedding_dim', 'config/l2',
       'config/learning_rate', 'config/max_epochs'],
      dtype='object')


,loss,val_loss,rprec train,rprec test,p@5 train,p@5 test,r@5 train,r@5 test,time_this_iter_s,should_checkpoint,...,training_iteration,trial_id,time_total_s,config/__trial_index__,config/batch_size,config/conv_layers,config/embedding_dim,config/l2,config/learning_rate,config/max_epochs
0,0.371526,0.506198,0.326488,0.149869,0.430769,0.196154,0.068064,0.097028,3.031254,True,...,46,3ea44134,211.610078,4,64,5,90,2.049287e-08,0.00014,50
1,0.381382,0.477144,0.331538,0.152820,0.425000,0.209615,0.075072,0.126507,6.682632,True,...,46,8c470f7f,285.687395,0,64,5,90,2.049287e-08,0.00014,50
3,0.383405,0.510567,0.316858,0.136350,0.419231,0.184615,0.069187,0.104465,4.128366,True,...,49,d3e40232,209.813512,3,64,5,90,2.049287e-08,0.00014,50
2,0.393274,0.554119,0.317956,0.159417,0.423077,0.209615,0.064125,0.108127,6.318547,True,...,34,9b123ff6,216.388431,2,64,5,90,2.049287e-08,0.00014,50
4,0.422135,0.531752,0.305300,0.129073,0.409615,0.184615,0.070582,0.097591,6.241138,True,...,23,dbaed356,141.478348,1,64,5,90,2.049287e-08,0.00014,50


In [4]:
# Group by config (except trial index) and do the mean
non_config_columns = ['config/__trial_index__', 'config/max_epochs']
cfg_columns = [c for c in rdf.columns if c.startswith('config/') and c not in non_config_columns]
non_results_cols = ['should_checkpoint', 'done', 'config/__trial_index__'] + non_config_columns
rdfgb = rdf.drop(columns=non_results_cols).groupby(cfg_columns).mean(numeric_only=True).reset_index()
print(len(rdfgb), "configurations were tested")
rdfgb.sort_values('p@5 test', ascending=False).tail(30)

1 configurations were tested


,config/batch_size,config/conv_layers,config/embedding_dim,config/l2,config/learning_rate,loss,val_loss,rprec train,rprec test,p@5 train,p@5 test,r@5 train,r@5 test,time_this_iter_s,training_iteration,time_total_s
0,64,5,90,2.049287e-08,0.00014,0.390345,0.515956,0.319628,0.145506,0.421538,0.196923,0.069406,0.106743,5.280387,39.6,212.995553
